In [1]:
# Importing important libraries
import numpy as np 
import pandas as pd
import pymysql
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Imporitng the dataset 
df_applicant = pd.read_json("C:/Users/jaipu/Documents/Black_Elephant/Weekend_January/Data_Engineering_Mini_project/applicant_info.json", lines = True)
df_financial = pd.read_json("C:/Users/jaipu/Documents/Black_Elephant/Weekend_January/Data_Engineering_Mini_project/financial_info.json", lines = True)
df_loan = pd.read_json("C:/Users/jaipu/Documents/Black_Elephant/Weekend_January/Data_Engineering_Mini_project/loan_info.json", lines = True)

In [5]:
# Pre-Process the data
def clean_df(df):
    df = df.replace({np.nan: None})
    return df

In [6]:
# Driver Code 
df_applicant = clean_df(df_applicant)
df_financial = clean_df(df_financial)
df_loan = clean_df(df_loan)

In [8]:
# Connecting to MySQL 
conn = pymysql.connect(
    host = 'localhost',
    user = 'root', # Give your own username here
    password = 'demo@123', # Give your own password here
    autocommit = True
)
cursor = conn.cursor()
print(cursor)

In [9]:
# Getting the structure of the tables 
df_applicant.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed
0,LP001002,Male,No,0,Graduate,No
1,LP001003,Male,Yes,1,Graduate,No
2,LP001005,Male,Yes,0,Graduate,Yes
3,LP001006,Male,Yes,0,Not Graduate,No
4,LP001008,Male,No,0,Graduate,No


In [10]:
# Getting the structure of the tables 
df_financial.head()

,Loan_ID,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,LP001002,5849,0.0,None,360.0,1.0
1,LP001003,4583,1508.0,128.0,360.0,1.0
2,LP001005,3000,0.0,66.0,360.0,1.0
3,LP001006,2583,2358.0,120.0,360.0,1.0
4,LP001008,6000,0.0,141.0,360.0,1.0


In [11]:
# Getting the structure of the tables 
df_loan.head()

,Loan_ID,Property_Area,Loan_Status
0,LP001002,Urban,Y
1,LP001003,Rural,N
2,LP001005,Urban,Y
3,LP001006,Urban,Y
4,LP001008,Urban,Y


In [ ]:
# "Loan_ID" is the common column between the 3 tables.

In [12]:
# Creating the database 
cursor.execute("CREATE DATABASE IF NOT EXISTS loan_db;")
cursor.execute("USE loan_db;")

0

In [13]:
# Creating the structure of the tables 
cursor.execute("""
CREATE TABLE IF NOT EXISTS applicant_info (
    Loan_ID VARCHAR(50) PRIMARY KEY,
    Gender VARCHAR(10),
    Married VARCHAR(10),
    Dependents VARCHAR(10),
    Education VARCHAR(20),
    Self_Employed VARCHAR(10)
)
""")

0

In [15]:
# Creating the structure of the tables 
cursor.execute("""
CREATE TABLE IF NOT EXISTS financial_info (
    Loan_ID VARCHAR(50) PRIMARY KEY,
    ApplicantIncome FLOAT,
    CoapplicantIncome FLOAT,
    LoanAmount FLOAT,
    Loan_Amount_Term FLOAT,
    Credit_History FLOAT
)
""")

0

In [17]:
# Creating the structure of the tables 
cursor.execute("""
CREATE TABLE IF NOT EXISTS loan_info (
    Loan_ID VARCHAR(50) PRIMARY KEY,
    Property_Area VARCHAR(20),
    Loan_Status VARCHAR(10)
)
""")

0

In [18]:
# Function for Inserting the data 
def insert_data(df, table_name):
    df = df.where(pd.notnull(df), None) # Replace NaN -> None
    cols = ",".join([f"{col}" for col in df.columns])
    placeholders = ",".join(["%s"] * len(df.columns))
    insert_sql = f"REPLACE INTO {table_name} ({cols}) VALUES ({placeholders})"

    clean_data = []
    for row in df.itertuples(index = False, name = None):
        clean_row = [None if (isinstance(val, float) and np.isnan(val)) else val for val in row]
        clean_data.append(tuple(clean_row))

    cursor.executemany(insert_sql, clean_data)
    conn.commit()

In [19]:
# Driver Code
insert_data(df_applicant, "applicant_info")
insert_data(df_financial, "financial_info")
insert_data(df_loan, "loan_info")

In [20]:
# Closing the cursor and the connection 
cursor.close()
conn.close()